<a href="https://colab.research.google.com/github/Ignacioelamo/LLMs4Phishing/blob/main/repharsing_data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
repo_url = "https://github.com/Ignacioelamo/LLMs4Phishing"
!git clone {repo_url}
!cd LLMs4Phishing

fatal: destination path 'LLMs4Phishing' already exists and is not an empty directory.


In [13]:
#!pip install transformers[torch]

In [14]:
import sys

# Añadir los directorio fuentes al path de Python
sys.path.append('/content/LLMs4Phishing/src')


# Verificar que se han añadido correctamente
print(sys.path)

import pandas as pd
from transformers import pipeline
import torch
from tqdm import tqdm

# Importing DataAugmenter directly from the file
#from data import DatasetAugmenter
#from rephrasers import EmailRephraser, ZeroShotRephraser, FewShotRephraser

['/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/root/.ipython', '/content/LLMs4Phishing/src', '/usr/local/lib/python3.11/dist-packages/setuptools/_vendor', '/tmp/tmpogc0p2wm', '/content/LLMs4Phishing/src', '/content/LLMs4Phishing/src']


# Código Python

In [15]:
import pandas as pd

class DatasetAugmenter:
    def __init__(self, dataset_path, rephraser):
        self.dataset = pd.read_csv(dataset_path)
        self.rephraser = rephraser
        self.augmented_data = []

    def augment_dataset(self, output_path, mode="zero_shot"):
        """Augment dataset with rephrased emails and show progress with tqdm."""
        self.augmented_data = []

        for _, row in tqdm(self.dataset.iterrows(), total=len(self.dataset), desc="Rephrasing Emails"):
            if mode == "zero_shot":
                prompt = self.rephraser.generate_zero_shot_prompt(row)
            else:
                raise ValueError("Invalid mode. Choose 'zero_shot'.")

            rephrased_email = self.rephraser.rephrase_email(prompt)
            subject = rephrased_email.split("Subject: ")[1].split("\n")[0] if "Subject: " in rephrased_email else "No Subject"
            body = rephrased_email.split("Body: ")[1] if "Body: " in rephrased_email else "No Body"
            self.augmented_data.append({
                "subject": subject,
                "sender": row['sender'],
                "receiver": row['receiver'],
                "body": body
            })

        augmented_df = pd.DataFrame(self.augmented_data)
        augmented_df.to_csv(output_path, index=False)
        print(f"Augmented dataset saved to {output_path}")

    def print_augmented_emails(self, n=1):
        """Print the first n augmented emails."""
        for i in range(n):
            print(f"Original email {i+1}:")
            print("Subject:", self.dataset.iloc[i]['subject'])
            print("Body:", self.dataset.iloc[i]['body'])
            print("\nAugmented email:")
            print("Subject:", self.augmented_data[i]['subject'])
            print("Body:", self.augmented_data[i]['body'])
            print("\n" + "-"*50 + "\n")

In [16]:
import pandas as pd
from transformers import pipeline, AutoModel, AutoTokenizer, AutoConfig
from transformers import AutoModelForCausalLM, AutoModelForSeq2SeqLM, AutoModelForMaskedLM
import torch

from abc import ABC, abstractmethod

class EmailRephraser(ABC):
    def __init__(self, model_name):
        self.model_name = model_name

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.generator = pipeline('text2text-generation', model=self.model_name, device=device)

    def rephrase_email(self, prompt, max_length=512):
        output = self.generator(prompt, max_length=max_length, num_return_sequences=1)
        return output[0]['generated_text']


In [17]:
from rephrasers.email_rephraser import EmailRephraser

class FewShotRephraser(EmailRephraser):
    def __init__(self, model_name, few_shot_examples):
        super().__init__(model_name)
        self.few_shot_examples = few_shot_examples

    def generate_few_shot_prompt(self, email_data):
        """Generate a few-shot prompt for rephrasing."""
        prompt = f"""
        Below are some examples of rephrased phishing emails. Please rephrase the following email in a similar way.

        Examples:
        {self.few_shot_examples}

        Original Email:
        Subject: {email_data['subject']}
        Sender: {email_data['sender']}
        Receiver: {email_data['receiver']}
        Body: {email_data['body']}

        Rephrased Email:
        """
        return prompt

In [18]:
from rephrasers.email_rephraser import EmailRephraser

class ZeroShotRephraser(EmailRephraser):
    def __init__(self, model_name):
        super().__init__(model_name)

    def generate_zero_shot_prompt(self, email_data):
        """Generate a zero-shot prompt for rephrasing."""
        prompt = f"""
        I want to train my machine learning model for my research on email phishing detection. I need you to rephrase the
        emails below and give me the new subject and body that are designed to bypass email phishing detectors. Keep the
        same sender and receiver information, avoid any sense of urgency or words like ’urgent’ and ’immediate’ in the body
        or subject, and avoid using deadlines or ultimatums. Avoid generic greetings, use the receiver information to greet them
        properly, and don’t mention sums of money or dollar amounts to make the email more legitimate. Ask for the same
        information the original email is asking for, just make the context more legitimate and keep the same core topic.

        Original Email:
        Subject: {email_data['subject']}
        Sender: {email_data['sender']}
        Receiver: {email_data['receiver']}
        Body: {email_data['body']}

        Rephrased Email:
        """
        return prompt

In [32]:
import pandas as pd

class DatasetAugmenter:
    def __init__(self, dataset_path, rephraser):
        self.dataset = pd.read_csv(dataset_path)
        self.rephraser = rephraser
        self.augmented_data = []

    def augment_dataset(self, output_path, mode="zero_shot"):
        """Augment dataset with rephrased emails and show progress with tqdm."""
        self.augmented_data = []

        for _, row in tqdm(self.dataset.iterrows(), total=len(self.dataset), desc="Rephrasing Emails"):
            if mode == "zero_shot":
                prompt = self.rephraser.generate_zero_shot_prompt(row)
            else:
                raise ValueError("Invalid mode. Choose 'zero_shot'.")

            rephrased_email = self.rephraser.rephrase_email(prompt)
            #print(f"\n\n{rephrased_email}")
            subject = rephrased_email.split("Subject: ")[1].split("\n")[0] if "Subject: " in rephrased_email else "No Subject"
            body = rephrased_email.split("Body: ")[1] if "Body: " in rephrased_email else "No Body"
            self.augmented_data.append({
                "subject": subject,
                "sender": row['sender'],
                "receiver": row['receiver'],
                "body": body
            })

        augmented_df = pd.DataFrame(self.augmented_data)
        augmented_df.to_csv(output_path, index=False)
        print(f"Augmented dataset saved to {output_path}")

    def print_augmented_emails(self, n=1):
        """Print the first n augmented emails."""
        for i in range(n):
            print(f"Original email {i+1}:")
            print("Subject:", self.dataset.iloc[i]['subject'])
            print("Body:", self.dataset.iloc[i]['body'])
            print("\nAugmented email:")
            print("Subject:", self.augmented_data[i]['subject'])
            print("Body:", self.augmented_data[i]['body'])
            print("\n" + "-"*50 + "\n")

# PRUEBAS

In [30]:
dataset_path = "./LLMs4Phishing/data/data.csv"

#Leeme el dataset sin la columna index
df = pd.read_csv(dataset_path)

#Quedate con las columnas subject, sender, receiver, body
df = df[["subject", "sender", "receiver", "body"]]

df = df.head(100)
df.to_csv("small_dataset.csv", index=False)
dataset_path = "./small_dataset.csv"


In [35]:
model_name = "google/flan-t5-xxl"  # Replace with your desired model
zero_shot_rephraser = ZeroShotRephraser(model_name)

    # Initialize dataset augmenter
augmenter = DatasetAugmenter(dataset_path, zero_shot_rephraser)

    # Augment dataset using zero-shot prompting
augmenter.augment_dataset("augmented_dataset_zero_shot.csv", mode="zero_shot")

config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/9.60G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cuda


OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 56.12 MiB is free. Process 2881 has 14.68 GiB memory in use. Of the allocated memory 14.45 GiB is allocated by PyTorch, and 115.18 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [34]:
augmenter.print_augmented_emails(n=100)

Original email 1:
Subject: Microsoft account unusual signin activity
Body: Microsoft account Unusual sign.in activity We detected something unusual about a recent sign-in to the Microsoft account [NAME] mailto:?&cc=&Subject=Report+The+User . Sign-in details Country/region: Russia/Moscow IP address: 103.225.77.255 Date: Fri, 08 Sep 2023 +0000 Platform: Windows 10 Browser: Firefox A user from Russia/Moscow just logged into your account from a new device, If this wasn't you, please report the user. If this was you, we'll trust similar activity in the future. Report The User mailto:?&cc=&subject=unusual signin activity&body=Report The User To opt out or change where you receive security notifications, click here mailto:?&cc=&Subject=Unsubscribe+me . Thanks, The Microsoft account team http://thebandalisty.com/track/o43062rdzGz18708448Gdrw1821750fYo33632dSjh176

Augmented email:
Subject: Microsoft account unusual signin activity
Body: No Body

------------------------------------------------